# Tavily Search

[Tavily's Search API](https://tavily.com) is a search engine built specifically for AI agents (LLMs), delivering real-time, accurate, and factual results at speed.

## Overview

### Integration details
| Class | Package | Serializable | [JS support](https://js.langchain.com/v0.2/docs/integrations/tools/tavily_search) |  Package latest |
| :--- | :--- | :---: | :---: | :---: |
| [TavilySearchResults](https://api.python.langchain.com/en/latest/tools/langchain_community.tools.tavily_search.tool.TavilySearchResults.html) | [langchain-community](https://api.python.langchain.com/en/latest/community_api_reference.html) | ❌ | ✅ |  ![PyPI - Version](https://img.shields.io/pypi/v/langchain-community?style=flat-square&label=%20) |

### Tool features
| [Returns artifact](/docs/how_to/tool_artifacts/) | Native async | Return data | Pricing |
| :---: | :---: | :---: | :---: |
| ✅ | ✅ | Title, URL, content, answer | 1,000 free searches / month | 


## Setup

The integration lives in the `langchain-community` package. We also need to install the `tavily-python` package.

In [ ]:
%pip install -qU "langchain-community>=0.2.11" tavily-python

### Credentials

We also need to set our Tavily API key. You can get an API key by visiting [this site](https://app.tavily.com/sign-in) and creating an account.

In [2]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability:

In [3]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Instantiation

Here we show how to instatiate an instance of the Tavily search tools, with 

In [1]:
from langchain_community.tools import TavilySearchResults

tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # include_domains=[...],
    # exclude_domains=[...],
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)

## Invocation

### [Invoke directly with args](/docs/concepts/#invoke-with-just-the-arguments)

The `TavilySearchResults` tool takes a single "query" argument, which should be a natural language query:

In [2]:
tool.invoke({"query": "What happened at the last wimbledon"})

[{'url': 'https://www.nytimes.com/athletic/5636272/2024/07/14/carlos-alcaraz-novak-djokovic-wimbledon-final/',
  'content': 'WIMBLEDON — Carlos Alcaraz beat Novak Djokovic in the Wimbledon final at the All England Club 6-2, 6-2, 7-6 on Sunday. The No 3 seed prevailed over the No 2 seed in a one-sided victory ...'},
 {'url': 'https://sports.yahoo.com/live/wimbledon-mens-semifinals-carlos-alcaraz-novak-djokovic-set-to-face-off-in-wimbledon-final-rematch-121511865.html',
  'content': "Carlos Alcaraz defeated Daniil Medvedev in Friday's first men's semifinal at Wimbledon, 6-7, 6-3, 6-4, 6-4. It's the second straight year that Alcaraz has knocked Medvedev out in the semifinals at ..."},
 {'url': 'https://www.cnn.com/2024/07/09/sport/novak-djokovic-wimbledon-crowd-quarterfinals-spt-intl/index.html',
  'content': 'Novak Djokovic produced another impressive performance at Wimbledon on Monday to cruise into the quarterfinals, but the 24-time grand slam champion was far from happy after his win.

### [Invoke with ToolCall](/docs/concepts/#invoke-with-toolcall)

We can also invoke the tool with a model-generated ToolCall, in which case a ToolMessage will be returned:

In [3]:
# This is usually generated by a model, but we'll create a tool call directly for demon purposes.
model_generated_tool_call = {
    "args": {"query": "euro 2024 host nation"},
    "id": "1",
    "name": "tavily",
    "type": "tool_call",
}
tool.invoke(model_generated_tool_call)

ToolMessage(content='[{"url": "https://www.radiotimes.com/tv/sport/football/euro-2024-location/", "content": "Euro 2024 host cities. Germany have 10 host cities for Euro 2024, topped by the country\'s capital Berlin. Berlin. Cologne. Dortmund. Dusseldorf. Frankfurt. Gelsenkirchen. Hamburg."}, {"url": "https://footballgroundguide.com/euro-2024-host-nation", "content": "As the Euro 2024 host nation, Germany will play the opening match against Scotland on 14 June before clashing with Hungary and finally Switzerland in the group stages. It is not an easy group but Germany is the highest-ranked by FIFA in group A of the Euro 2024. Apart from European Championship access, Germany has also been a powerhouse in ..."}, {"url": "https://www.sportingnews.com/us/soccer/news/where-euro-2024-host-nation-next-european-championships/fguxslt9krlghuf1rcn9yf4v", "content": "Germany will host Euro 2024, the next men\'s European Championships, from June 14 to July 14, 2024. Find out the 10 stadiums, the sc

## Chaining

We can use our tool in a chain by first binding it to a [tool-calling model](/docs/how_to/tool_calling/) and then calling it:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [18]:
# | output: false
# | echo: false

# !pip install -qU langchain langchain-openai
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="gpt-4o", model_provider="openai", temperature=0)

In [19]:
import datetime

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

today = datetime.datetime.today().strftime("%D")
prompt = ChatPromptTemplate(
    [
        ("system", f"You are a helpful assistant. The date today is {today}."),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

# specifying tool_choice will force the model to call this tool.
llm_with_tools = llm.bind_tools([tool])

llm_chain = prompt | llm_with_tools


@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = tool.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


tool_chain.invoke("who won the last womens singles wimbledon")

AIMessage(content="Barbora Krejcikova won the women's singles title at Wimbledon 2024. She defeated Jasmine Paolini in the final with a score of 6-2, 2-6, 6-4. This victory marked Krejcikova's second Grand Slam singles title.", response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 663, 'total_tokens': 723}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_4e2b2da518', 'finish_reason': 'stop', 'logprobs': None}, id='run-10165060-e69e-4c66-bae9-7f93776dd7ff-0', usage_metadata={'input_tokens': 663, 'output_tokens': 60, 'total_tokens': 723})

Here's the [LangSmith trace](https://smith.langchain.com/public/2e9dd540-0bbc-4450-ba3e-d49bf2c7819d/r) for this run.

## API reference

For detailed documentation of all TavilySearchResults features and configurations head to the API reference: https://api.python.langchain.com/en/latest/tools/langchain_community.tools.tavily_search.tool.TavilySearchResults.html